In [0]:
pip install xgboost

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#%restart_python
from pyspark.sql import SparkSession
#from sparkxgb import SparkXGBRegressor  # Import XGBoost for PySpark
from xgboost.spark import SparkXGBRegressor as sparkxgb
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
# 1️⃣ Initialize Spark Session
#spark = SparkSession.builder.appName(PySparkXGBoostRegression).getOrCreate()

# 2️⃣ Load Dataset (California Housing dataset)
#data = spark.read.csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv",
#                      header=True, inferSchema=True)

from pyspark.sql import SparkSession
#from sparkxgb import SparkXGBRegressor  # Import XGBoost for PySpark
from xgboost.spark import SparkXGBRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

data = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/FileStore/tables/housing.csv")
#data = spark.read.format("csv").option("header", "true").load("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")
# 3️⃣ Check Schema
data.printSchema()

# 4️⃣ Handle Missing Values (Drop Nulls)
data = data.dropna()

# 5️⃣ Feature Engineering - Assemble Features into a Single Column
feature_columns = data.columns[:-1]  # Exclude target column (median_house_value)
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data).select("features", "median_house_value")

# 6️⃣ Train-Test Split (80-20)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# 7️⃣ Initialize & Train XGBoost Regressor
xgb = SparkXGBRegressor(features_col="features", label_col="median_house_value",
                        maxDepth=6, eta=0.1, objective="reg:squarederror")
model = xgb.fit(train_data)

# 8️⃣ Model Evaluation on Test Data
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="median_house_value", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# 9️⃣ Print Model Performance
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



2025-02-05 07:43:12,669 INFO XGBoost-PySpark: _fit Running xgboost-2.1.3 on 1 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'maxDepth': 6, 'eta': 0.1, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2025-02-05 07:43:20,320 INFO XGBoost-PySpark: _fit Finished xgboost training!


Root Mean Squared Error (RMSE): 966.74


In [0]:
data = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/FileStore/tables/housing.csv")

data.describe(["median_house_value"]).show()


+-------+------------------+
|summary|median_house_value|
+-------+------------------+
|  count|             20640|
|   mean|206855.81690891474|
| stddev|115395.61587441359|
|    min|           14999.0|
|    max|          500001.0|
+-------+------------------+

